# Aufbereitung der Daten

Um aus den im [vorherigen Kapitel](./00_Scraping_Crawling_Beitraege_Kommentare.ipynb) gescrapten Daten nun aufschlussreiche Analysen erstellen und Informationen extrahieren zu können, müssen diese jedoch erst aufbereitet werden. Im Folgenden werden also Texte durch das Entfernen von störendem Rauschen in eine saubere Form verwandelt.

<hr/>

In [1]:
%run "settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

# to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sqlite3
import pandas as pd
import re
import spacy
import textacy

In [3]:
def pd_read_sql(database, sql):
    with sqlite3.connect(database) as con:
        dataframe = pd.read_sql_query(sql, con)
        return dataframe

def pd_write_sql(database, table, df):
    with sqlite3.connect(database) as con:
        df.to_sql(table, con, index=False, if_exists='replace')

In [4]:
df = pd_read_sql("data.sqlite", "SELECT * FROM comments")

Wie man hier schön erkennen kann, enthält der DataFrame in der Spalte "body" die rohen, unaufbereiteten Daten mit störendem Rauschen und Artefakten wie Links, ">"-Zeichen und "\n" (Zeilenumbrüche) oder ähnlichem, die für die Analysen erstmal wenig nützlich sind. 
Diese gilt es im Folgenden nun zu bereinigen.

Wir verwenden hier [spaCy](https://spacy.io/), die eine NLP Pipeline mit Verarbeitungskomponenten wie einen Tokenizer, einen Tagger, einen Parser und einen NER-Detektor zur Verfügung stellt. Da wir uns aber bei unserer Cyberpunk Analyse auf die Meinung der Nutzer zum Spiel an sich und nicht auf die Klassifikation von vordefinierten Kategorien wie z.B. Personennamen, Organisationen, Orte und andere im Spiel vorkommende Entitäten fokussieren wollen, arbeiten wir ohne die NER- und Parser-Komponente.

<div><img src="https://spacy.io/pipeline-fde48da9b43661abcdf62ab70a546d71.svg" width="600" /></div>

Um die Verarbeitung auf der GPU laufen zu lassen (falls vorhanden), wird spaCy instruiert, dies auch zu tun. Falls nicht, wird automatisch die CPU verwendet.

In [5]:
if spacy.prefer_gpu():
    print("Working on GPU.")
else:
    print("No GPU found, working on CPU.")

nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])
nlp.pipeline

Working on GPU.


[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1f05a127630>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1f04607f220>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1f06edaaa00>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1f06edaa100>)]

Zitiert man einen Reddit Kommentar, entsteht im Markdown ein ">" Zeichen. Da aber dieses Zeichen ebenso im Text als Vergleichszeichen vorkommen kann, markieren wir das Zeichen als Satzzeichen, um es so herausfiltern zu können.

In [6]:
nlp.vocab['>'].is_punct = True

Im Folgenden definieren wir nun eine Funktion, welche Texte übergeben bekommt und mit regulären Ausdrücken nach Unreinheiten sucht und diese ersetzt. So werden beispielsweise Links auf andere Beiträge und unnötige Zeichen effektiv herausgefiltert.


Mit der zweiten Funktion ist es möglich, nachdem wir mit der NLP-Pipeline aus einem Text ein `doc`-Objekt erzeugt haben, die entsprechenden Eigenschaften der Tokens mitsamt den Attributen darzustellen.


In [7]:
def clean(text):
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    # remove * character
    text = re.sub(r'\*', '', text)
    # remove . or ... characters
    text = re.sub(r'\.+', '', text)
    #remove links like (https:...) or https:....
    text = re.sub(r'\(*https:[^\\(\)\n ]*\)*', '', text)
    return text.strip()

def display_nlp(doc, include_punct=False):
    """Generate data frame for visualization of spaCy tokens."""
    rows = []
    for i, t in enumerate(doc):
        if not t.is_punct or include_punct:
            row = {'token': i,  'text': t.text, 'lemma_': t.lemma_,
                   'is_stop': t.is_stop, 'is_punct': t.is_punct,
                   'pos_': t.pos_, 'morph': t.morph}
            rows.append(row)
    
    df = pd.DataFrame(rows).set_index('token')
    df.index.name = None
    return df

Folglich wird neben der body-Spalte, welche den Rohtext enthält, eine weitere Spalte erstellt, die die bereinigten Texte erfasst.

In [8]:
df.insert(3, 'clean_body', '')
df['clean_body'] = df['body'].progress_map(clean)

  0%|          | 0/441494 [00:00<?, ?it/s]

Vergleicht man nun die body-Spalte mit der clean_body-Spalte, kann man bereits einen signifikanten Unterschied erkennen: Die Leserlichkeit hat sich um einiges verbessert.

In [9]:
df[["body", "clean_body"]].sample(5)

,body,clean_body
109584,Feels like GTA meets Deus ex to me. Driving missions and delivery missions are so gta,Feels like GTA meets Deus ex to me Driving missions and delivery missions are so gta
179906,Yakuza minigame ♥,Yakuza minigame ♥
41782,Hey it's how Bethesda's operated for the past 15 some-odd years.,Hey it's how Bethesda's operated for the past 15 some-odd years
43937,Bruh that sucks,Bruh that sucks
43403,Isn't pokemon a triple A open world RPG too?,Isn't pokemon a triple A open world RPG too?


Um die NLP-Pipeline zu demonstrieren, wird hier ein Beispieltext aus der Spalte "clean_body" geladen.

In [10]:
text = df["clean_body"].sample().iloc[0]
text

'Broken down? Watch the video again, and even if, then just driving away is still ab big asshole move'

Durch die NLP-Pipeline wird, wie oben beschrieben, ein Doc-Objekt erzeugt, dessen Eigenschaften mit Hilfe der `display_nlp` Funktion ausgegeben werden. 
  * text: der originale Text
  * lemma_: grammatikalische Grundform
  * is_stop: Flag, ob Stopp-Wort
  * is_punct: Flag, ob Satzzeichen
  * pos_: Part-of-Speech-Tag
  * morph: Weitere morphologische Eigenschaften

In [11]:
doc = nlp(text)
display_nlp(doc)

,text,lemma_,is_stop,is_punct,pos_,morph
0,Broken,break,False,False,VERB,"(Aspect=Perf, Tense=Past, VerbForm=Part)"
1,down,down,True,False,ADP,()
3,Watch,watch,False,False,VERB,(VerbForm=Inf)
4,the,the,True,False,DET,"(Definite=Def, PronType=Art)"
5,video,video,False,False,NOUN,(Number=Sing)
6,again,again,True,False,ADV,()
8,and,and,True,False,CCONJ,(ConjType=Cmp)
9,even,even,True,False,ADV,()
10,if,if,True,False,SCONJ,()
12,then,then,True,False,ADV,(PronType=Dem)


## Aufbereitung aller Redditkommentare ##

Nun kann die massenhafte Aufbereitung der Redditkommentare beginnen. Dafür definieren wir uns eine Funktion, mit der wir die Token nach bestimmten Eigenschaften einfach aus dem Doc-Objekt extrahieren können.

In [12]:
def extract_lemmas(doc, **kwargs):
    return [t.lemma_ for t in textacy.extract.words(doc, **kwargs)]

Da wir bei unserer Analyse zu Cyperpunk 2077 speziell die User-Meinung zum Spiel untersuchen möchten, sind vorallem Adjektive und Verben zur Bestimmung des Sentiments ([siehe Notebook 03 Sentiment Analyse und Topic Analyse](./03_Sentiment_und_Topic_Analyse.ipynb)) von großer Bedeutung.

Deshalb werden durch die `extract_nlp` Funktion Lemmas jeweils in eine Spalte für Adjektive/Verben, Nomen und Lemmas aufgeteilt.

In [13]:
def extract_nlp(doc):
    return {
    'lemmas'     : extract_lemmas(doc,exclude_pos = ['PART', 'PUNCT', 'DET', 'PRON', 'SYM', 'SPACE'], filter_stops = False),
    'adjs_verbs' : extract_lemmas(doc, include_pos = ['ADJ', 'VERB']),
    'nouns'      : extract_lemmas(doc, include_pos = ['NOUN', 'PROPN']),
    }

Angewendet auf einen Beispielkommentar wird der Text wie folgt aufgeteilt:

In [14]:
text_example = df.sample(1)["clean_body"].iloc[0]
print(text_example+"\n")

doc_example = nlp(text_example)
pp.pprint(extract_nlp(doc_example))

nope, im playing in vr in front of a green screen 24/7, try again

{'adjs_verbs': ['m', 'play', 'green', 'try'],
 'lemmas': ['nope',
            'm',
            'play',
            'in',
            'vr',
            'in',
            'front',
            'of',
            'green',
            'screen',
            '24/7',
            'try',
            'again'],
 'nouns': ['vr', 'screen']}


In [15]:
display_nlp(doc_example)

,text,lemma_,is_stop,is_punct,pos_,morph
0,nope,nope,False,False,INTJ,()
2,i,I,True,False,PRON,"(Case=Nom, Number=Sing, Person=1, PronType=Prs)"
3,m,m,False,False,VERB,"(Tense=Pres, VerbForm=Fin)"
4,playing,play,False,False,VERB,"(Aspect=Prog, Tense=Pres, VerbForm=Part)"
5,in,in,True,False,ADP,()
6,vr,vr,False,False,NOUN,(Number=Sing)
7,in,in,True,False,ADP,()
8,front,front,True,False,NOUN,(Number=Sing)
9,of,of,True,False,ADP,()
10,a,a,True,False,DET,"(Definite=Ind, PronType=Art)"


Jetzt haben wir alle Funktionen deklariert, um die massenhafte Datenaufbereitung für all unsere gescrapten Redditkommentare zu starten.

Dafür legen wir zuerst die zusätzlichen Spalten in unserem Ziel-DataFrame an.

In [16]:
nlp_columns = list(extract_nlp(nlp.make_doc('')).keys())
print(nlp_columns)

for col in nlp_columns:
    df[col] = None

['lemmas', 'adjs_verbs', 'nouns']


**Achtung:** Die Aufbereitung der insgesamt knapp 442.000 Kommentare kann je nach Hardware bis zu 2 Stunden brauchen. Folgender Abschnitt kann also übersprungen werden, da die fertig aufbereiteten Kommentare bereits in der Datenbank gespeichert sind. Weiter geht es dann mit [Kapitel 02 Explorative Datenanalyse](./02_Explorative_Datenanalyse.ipynb).

Wir verarbeiten im Folgenden jeweils 100 Datensätze der Redditkommentare auf einmal und fügen die extrahierten Lemmas jeweils an das Ende des Dataframes an.

In [17]:
batch_size = 100
num_batches = math.ceil(len(df) / batch_size)

for i in tqdm(range(0, len(df), batch_size), total=num_batches):
    
    # spaCy Batch-Verarbeitung mit nlp.pipe, liefert eine Liste von Ergebnis-Docs
    docs = nlp.pipe(df['clean_body'][i:i+batch_size])
    
    # Extraktion der Lemmas und Eintragen im DataFrame für einen Batch
    for j, doc in enumerate(docs):
        for col, values in extract_nlp(doc).items():
            df[col].iloc[i+j] = values

  0%|          | 0/4415 [00:00<?, ?it/s]

In [18]:
for column in nlp_columns:
    df[column] = df[column].progress_map(lambda tokens: " ".join(tokens))
df[nlp_columns].head(5)

  0%|          | 0/441494 [00:00<?, ?it/s]

  0%|          | 0/441494 [00:00<?, ?it/s]

  0%|          | 0/441494 [00:00<?, ?it/s]

,lemmas,adjs_verbs,nouns
0,post have receive multiple report and have be tempoarily remove until moderator can review be bot and action be perform automatically please contact moderator of subreddit if have question or concern,receive multiple remove review perform contact,post report moderator bot action moderator subreddit question concern
1,ahhhh plan bet will go great,bet great,plan
2,just fire up rdr2 again few day ago and have same thought great game,fire great,rdr2 day thought game
3,remember when CD Projekt Red say that cyberpunk 2077 would be as polished as Red Dead redemption 2 sadly know be true statement,remember say polished know true,CD Projekt Red cyberpunk Red Dead redemption statement
4,just way npc mindlessly exist in cyberpunk completely ruin for and ’ idea be make NPC outfit 100 lime green neon jumpsuit outfit stand out like crazy when ’ 17 people and 11 of be wear same exact ...,exist ruin green stand crazy wear exact implement enjoy static look,way npc cyberpunk idea NPC outfit lime neon jumpsuit outfit people outfit quest system cyberpunkI gunplay world game life


Letztlich bleibt nur noch das Speichern des erzeugten DataFrames in der Tabelle `comments_prepared` in der Datenbank`data.sqlite` übrig.

In [19]:
pd_write_sql("data.sqlite", "comments_prepared", df)

## Quellen ##

* https://spacy.io/
* https://spacy.io/usage/processing-pipelines
* https://spacy.io/usage/linguistic-features